In [18]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.multioutput import MultiOutputClassifier

In [19]:
# Importamos data cruda
data_recepcion = pd.read_csv('recepcion.csv')
data_donacion = pd.read_csv('donacion.csv')

# Concatenamos los datasets
X_recepcion = data_recepcion.iloc[:,:8]
y_recepcion = data_recepcion.iloc[:,8:]

X_donacion = data_donacion.iloc[:,:8]
y_donacion = data_donacion.iloc[:,8:]

# Dividimos en train y test
X_train_recepcion, X_test_recepcion, y_train_recepcion, y_test_recepcion = train_test_split(X_recepcion, y_recepcion, test_size=0.2, random_state=24)
X_train_donacion, X_test_donacion, y_train_donacion, y_test_donacion = train_test_split(X_donacion,y_donacion, test_size=0.2, random_state=24)

In [20]:
# Definimos el modelo
bosque = RandomForestClassifier()

# Entrenamos el modelo
modelo_recepcion = MultiOutputClassifier(bosque, n_jobs=-1)
modelo_recepcion.fit(X_train_recepcion, y_train_recepcion)

modelo_donacion = MultiOutputClassifier(bosque, n_jobs=-1)
modelo_donacion.fit(X_train_donacion, y_train_donacion)

# Prediciones
y_pred_recepcion = modelo_recepcion.predict(X_test_recepcion)
y_pred_donacion = modelo_donacion.predict(X_test_donacion)

y_pred_recepcion, y_pred_donacion


(array([[1, 1, 1, ..., 1, 1, 1],
        [1, 0, 0, ..., 0, 0, 0],
        [1, 0, 1, ..., 0, 1, 0],
        ...,
        [1, 1, 0, ..., 1, 0, 0],
        [1, 0, 0, ..., 0, 0, 0],
        [1, 1, 1, ..., 1, 1, 1]], dtype=int64),
 array([[0, 0, 0, ..., 0, 1, 1],
        [0, 0, 0, ..., 0, 0, 1],
        [0, 0, 0, ..., 0, 0, 1],
        ...,
        [0, 1, 0, ..., 1, 0, 1],
        [1, 1, 1, ..., 1, 1, 1],
        [1, 1, 1, ..., 1, 1, 1]], dtype=int64))

In [21]:
#verificamos precision
print(classification_report(y_test_recepcion, y_pred_recepcion), classification_report(y_test_donacion, y_pred_donacion))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00     10000
           1       1.00      1.00      1.00      5033
           2       1.00      1.00      1.00      4958
           3       1.00      1.00      1.00      2494
           4       1.00      1.00      1.00      4923
           5       1.00      1.00      1.00      2473
           6       1.00      1.00      1.00      2454
           7       1.00      1.00      1.00      1228

   micro avg       1.00      1.00      1.00     33563
   macro avg       1.00      1.00      1.00     33563
weighted avg       1.00      1.00      1.00     33563
 samples avg       1.00      1.00      1.00     33563
               precision    recall  f1-score   support

           0       1.00      1.00      1.00      1265
           1       1.00      1.00      1.00      2511
           2       1.00      1.00      1.00      2535
           3       1.00      1.00      1.00      5076
           4       1.00 

In [22]:
print(accuracy_score(y_test_recepcion, y_pred_recepcion), accuracy_score(y_test_recepcion, y_pred_recepcion))

1.0 1.0


In [23]:
import joblib

# Exportar el modelo entrenado
joblib.dump(modelo_recepcion, 'modelo_recepcion.pkl')
joblib.dump(modelo_donacion, 'modelo_donacion.pkl')

['modelo_donacion.pkl']